## Programming Analytics:  Healthcare Coverage in the U.S.A.
### Team Members:
* Yuttawee Kongtananan (GitHub ID: yuttk)
* Vel (Tien-Yun) Wu (GitHub ID: velwu)
* Eva (Yi-Ting) Huang (GitHub ID: iameva62948)

In [10]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["font.family"] = "fantasy"
import numpy as np
import random
import scipy
import scipy.misc
import scipy.cluster

from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as pgo
import plotly.offline as pol
import importlib
import ipywidgets

In [11]:
clas = importlib.import_module('central_analysis')
print("Import of central_analysis.py is successful")

Import of central_analysis.py is successful


## Dataset 1: Unemployment Rate

In [12]:
plt.clf()
year_slider = range(2009, 2019, 1)
data_toggle = ipywidgets.Checkbox(False, description='Show Data')
@ipywidgets.interact(end_year = year_slider, show_data = data_toggle)
def check_unployment(end_year = 2012, show_data = False):
    data_un = clas.read_unemployment_by_year(2008, end_year, True)
    if show_data:
        display(data_un)

<Figure size 432x288 with 0 Axes>

interactive(children=(Dropdown(description='end_year', index=3, options=(2009, 2010, 2011, 2012, 2013, 2014, 2…

## Dataset 2: Insurance Coverage of different Types and Years
- Different types of insurance have different coverages
- At the same time, a single type of insurance also has ebbs and flows thorughout the years

In [13]:
plt.clf()
insurance_type = ['Any coverage', 'Uninsured', 'Private', '..Employer-based', '..Direct-purchase', '..TRICARE', 'Public', '..Medicaid', '..Medicare', '..VA Care']
year_slider = ipywidgets.IntSlider(min= 2008,max=2018,step=1,value=2012, description='Year')
color_theme = ['Blackbody','Bluered','Blues','Earth','Electric','Greens','Greys','Hot','Jet','Picnic','Portland','Rainbow','RdBu','Reds','Viridis','YlGnBu','YlOrRd']

@ipywidgets.interact(insurance_type = insurance_type,
                     choice_of_year = year_slider, 
                     map_theme = color_theme)
def customized_choropleth(insurance_type = 'Any coverage', choice_of_year = 2012, map_theme = 'Earth'):
    demo_df_to_map = clas.read_health_care_coverage_by_year(2008, 2018, insurance_type, False)
    clas.spawn_choropleth_from_dataframe(demo_df_to_map, choice_of_year, str(insurance_type) + ' type Year ' + str(choice_of_year), map_theme, "% by state", True)

<Figure size 432x288 with 0 Axes>

interactive(children=(Dropdown(description='insurance_type', options=('Any coverage', 'Uninsured', 'Private', …

## Dataset 3: Household Income
-  We can pass the same dataset into different plots and switch between them efficiently.
- This is a demomonstration of the **modularity** and **optimization** of our functions which divide the tasks of getting data and plotting them.

In [14]:
plt.clf()
plot_type = ["Line Plot", "Choropleth Map"]
@ipywidgets.interact(plot_type = plot_type)
def choose_a_graph_for_HHI(plot_type = "Line Plot"):
    demo_df_to_map = clas.read_household_income_by_year(1984, 2018, False)
    if plot_type == "Line Plot":
        clas.spawn_line_plot_from_dataframe(demo_df_to_map, "Household Income (USD) by State and Years", "Years", "Household Income (USD)")
    if plot_type == "Choropleth Map":
        clas.spawn_choropleth_from_dataframe(demo_df_to_map, 2018, "Household Income (USD) by State Year 2018", "Greens", "% by state", True)

<Figure size 432x288 with 0 Axes>

interactive(children=(Dropdown(description='plot_type', options=('Line Plot', 'Choropleth Map'), value='Line P…

## Dataset 4 : CPI

In [15]:
demo_df_cpi = clas.read_cpi_by_year(2003, 2020)
demo_df_cpi

,Years,CPI All items,CPI Medical,CPI All items Growth,CPI Medical Growth
68,2003,282.712487,230.703265,0.022900,0.043289
69,2004,290.103915,240.503673,0.026145,0.042481
70,2005,300.261587,249.743089,0.035014,0.038417
71,2006,310.179259,253.052907,0.033030,0.013253
72,2007,319.028324,264.378149,0.028529,0.044754
73,2008,331.171117,275.691428,0.038062,0.042792
74,2009,329.985225,285.144522,-0.003581,0.034289
75,2010,335.103959,283.881984,0.015512,-0.004428
76,2011,345.417992,290.193227,0.030779,0.022232
77,2012,352.577801,299.527723,0.020728,0.032166


## Correlations:

In [16]:
demo_df_un = clas.read_unemployment_by_year(2008, 2018, False)
demo_df_hc = clas.read_health_care_coverage_by_year(2008, 2018, 'Private', False)
demo_df_hh_ic = clas.read_household_income_by_year(1991, 2018, False)

In [29]:
insurance_type = ['Any coverage', 'Uninsured', 'Private', '..Employer-based', '..Direct-purchase', '..TRICARE', 'Public', '..Medicaid', '..Medicare', '..VA Care']
growth_rate_var = ["None", "First", "Second", "Both"]
data_toggle = ipywidgets.Checkbox(False, description='Show Data')
plot_toggle = ipywidgets.Checkbox(True, description='Show Plot')

@ipywidgets.interact(insurance_type = insurance_type, corr_type = growth_rate_var, data_toggle = data_toggle, plot_toggle = plot_toggle)
def choose_a_method_for_corr(insurance_type = "Any coverage", corr_type = "None", data_toggle = False, plot_toggle = True):
    demo_df_hc = clas.read_health_care_coverage_by_year(2008, 2018, insurance_type, False)
    clas.merging_dataframes_on_years_plus_correlations(demo_df_hc, demo_df_hh_ic, "Healthcare coverage", "HH Income", 
                                                   corr_type, data_toggle, plot_toggle)

interactive(children=(Dropdown(description='insurance_type', options=('Any coverage', 'Uninsured', 'Private', …

In [30]:
insurance_type = ['Any coverage', 'Uninsured', 'Private', '..Employer-based', '..Direct-purchase', '..TRICARE', 'Public', '..Medicaid', '..Medicare', '..VA Care']
growth_rate_var = ["None", "First", "Second", "Both"]
data_toggle = ipywidgets.Checkbox(False, description='Show Data')
plot_toggle = ipywidgets.Checkbox(True, description='Show Plot')
@ipywidgets.interact(insurance_type = insurance_type, corr_type = growth_rate_var, data_toggle = data_toggle, plot_toggle = plot_toggle)
def choose_a_method_for_corr_2(insurance_type = "Any coverage", corr_type = "None", data_toggle = False, plot_toggle = True):
    demo_df_hc = clas.read_health_care_coverage_by_year(2008, 2018, insurance_type, False)
    clas.merging_dataframes_on_years_plus_correlations(demo_df_hc, demo_df_un, "Healthcare coverage", "Unemployment", corr_type, data_toggle, plot_toggle)


interactive(children=(Dropdown(description='insurance_type', options=('Any coverage', 'Uninsured', 'Private', …